# Ecommerce Analysis and Prediction

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_cust = pd.read_csv(r'C:\Users\Adriana\Documents\GitHub\ecommerce-analysis\data-ecomm\customers_dataset.csv')
data_geoloc = pd.read_csv(r'C:\Users\Adriana\Documents\GitHub\ecommerce-analysis\data-ecomm\geolocation_dataset.csv')
data_items = pd.read_csv(r'C:\Users\Adriana\Documents\GitHub\ecommerce-analysis\data-ecomm\order_items_dataset.csv')
data_payments = pd.read_csv(r'C:\Users\Adriana\Documents\GitHub\ecommerce-analysis\data-ecomm\order_payments_dataset.csv')
data_review = pd.read_csv(r'C:\Users\Adriana\Documents\GitHub\ecommerce-analysis\data-ecomm\order_reviews_dataset.csv')
data_order = pd.read_csv(r'C:\Users\Adriana\Documents\GitHub\ecommerce-analysis\data-ecomm\orders_dataset.csv')
data_product = pd.read_csv(r'C:\Users\Adriana\Documents\GitHub\ecommerce-analysis\data-ecomm\products_dataset.csv')
data_seller = pd.read_csv(r'C:\Users\Adriana\Documents\GitHub\ecommerce-analysis\data-ecomm\sellers_dataset.csv')
data_prod_trans = pd.read_csv(r'C:\Users\Adriana\Documents\GitHub\ecommerce-analysis\data-ecomm\product_category_name_translation.csv')

In [3]:
dataset = [data_cust,
           data_geoloc,
           data_items,
           data_payments,
           data_review,
           data_order,
           data_product,
           data_seller,
           data_prod_trans]
for i in dataset:
    shp = i.shape
    print(shp)

(99441, 5)
(1000163, 5)
(112650, 7)
(103886, 5)
(99224, 7)
(99441, 8)
(32951, 9)
(3095, 4)
(71, 2)


In [4]:
data_cust.head(1)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP


In [5]:
data_geoloc.head(1)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP


In [6]:
data_items.head(1)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29


In [7]:
data_payments.head(1)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33


In [8]:
data_review.head(1)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59


In [9]:
data_order.head(1)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00


In [10]:
data_seller.head(1)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP


In [11]:
data_product.head(1)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0


In [12]:
data_prod_trans.head(1)

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty


## Creating a Single dataset by merging

### data_prod_trans -> data_product -> data_items

In [13]:
data_product = data_product.merge(data_prod_trans, on='product_category_name')
data_product.drop('product_category_name', axis=1, inplace=True)
data_product.rename(columns={'product_category_name_english':'product_category_name'}, inplace=True)

In [14]:
data_items1 = data_items.merge(data_product, on='product_id')

### data_geoloc -> data_seller -> data_items

In [15]:
data_geoloc1 = data_geoloc.drop_duplicates(subset='geolocation_zip_code_prefix')

In [16]:
data_seller_merge = data_seller.merge(data_geoloc1, left_on='seller_zip_code_prefix',
                                      right_on='geolocation_zip_code_prefix', how='inner')
data_seller_merge = data_seller_merge.drop(columns=['geolocation_zip_code_prefix','geolocation_city','geolocation_state'])
data_seller_merge = data_seller_merge.rename(columns={'geolocation_lat':'seller_geolocation_lat',
                                                      'geolocation_lng':'seller_geolocation_lng'})

In [17]:
data_items2 = data_items1.merge(data_seller_merge, on='seller_id')

### data_geoloc -> data_cust -> data_order

In [18]:
data_cust_merge = data_cust.merge(data_geoloc1,left_on='customer_zip_code_prefix',
                           right_on='geolocation_zip_code_prefix', how='inner')
data_cust_merge = data_cust_merge.drop(columns=['geolocation_zip_code_prefix','geolocation_city','geolocation_state'])
data_cust_merge = data_cust_merge.rename(columns={'geolocation_lat':'customer_geolocation_lat',
                                                  'geolocation_lng':'customer_geolocation_lng'})

In [19]:
data_order1 = data_order.merge(data_cust_merge, on='customer_id')

### data_items -> data_order

In [20]:
data_order2 = data_order1.merge(data_items2, on='order_id')

### data_payments -> data_order

In [21]:
data_order3 = data_order2.merge(data_payments, on='order_id')

### data_review -> data_order

In [22]:
data_raw = data_order3.merge(data_review,on='order_id')

In [23]:
pd.set_option('display.max_column', None)
data_raw.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_geolocation_lat,customer_geolocation_lng,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,seller_zip_code_prefix,seller_city,seller_state,seller_geolocation_lat,seller_geolocation_lng,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.574809,-46.587471,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,9350,maua,SP,-23.680114,-46.452454,1,credit_card,1,18.12,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.574809,-46.587471,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,9350,maua,SP,-23.680114,-46.452454,3,voucher,1,2.00,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.574809,-46.587471,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,9350,maua,SP,-23.680114,-46.452454,2,voucher,1,18.59,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,-12.169860,-44.988369,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,29.0,178.0,1.0,400.0,19.0,13.0,19.0,perfumery,31570,belo horizonte,SP,-19.810119,-43.984727,1,boleto,1,141.46,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,-16.746337,-48.514624,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,46.0,232.0,1.0,420.0,24.0,19.0,21.0,auto,14840,guariba,SP,-21.362358,-48.232976,1,credit_card,3,179.12,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58


In [24]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115037 entries, 0 to 115036
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       115037 non-null  object 
 1   customer_id                    115037 non-null  object 
 2   order_status                   115037 non-null  object 
 3   order_purchase_timestamp       115037 non-null  object 
 4   order_approved_at              115023 non-null  object 
 5   order_delivered_carrier_date   113852 non-null  object 
 6   order_delivered_customer_date  112653 non-null  object 
 7   order_estimated_delivery_date  115037 non-null  object 
 8   customer_unique_id             115037 non-null  object 
 9   customer_zip_code_prefix       115037 non-null  int64  
 10  customer_city                  115037 non-null  object 
 11  customer_state                 115037 non-null  object 
 12  customer_geolocation_lat      

In [25]:
data_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
customer_zip_code_prefix,115037.0,34983.378296,29829.438971,1003.000000,11095.000000,24230.000000,58297.000000,99980.000000
customer_geolocation_lat,115037.0,-21.235765,5.571871,-36.605374,-23.590023,-22.929912,-20.198222,42.184003
customer_geolocation_lng,115037.0,-46.197857,4.050645,-72.666706,-48.101695,-46.632021,-43.625993,-8.577855
order_item_id,115037.0,1.194685,0.686686,1.000000,1.000000,1.000000,1.000000,21.000000
price,115037.0,120.650015,182.853114,0.850000,39.900000,74.900000,134.900000,6735.000000
freight_value,115037.0,20.049325,15.850423,0.000000,13.080000,16.320000,21.200000,409.680000
product_name_lenght,115037.0,48.754288,10.038336,5.000000,42.000000,52.000000,57.000000,76.000000
product_description_lenght,115037.0,785.867573,653.051492,4.000000,345.000000,600.000000,982.000000,3992.000000
product_photos_qty,115037.0,2.200318,1.713213,1.000000,1.000000,1.000000,3.000000,20.000000
product_weight_g,115036.0,2112.365746,3776.693083,0.000000,300.000000,700.000000,1800.000000,40425.000000


In [26]:
data_raw.select_dtypes(include='object').describe().T

,count,unique,top,freq
order_id,115037,96036,895ab968e7bb0d5659d16cd74cd1650c,63
customer_id,115037,96036,270c23a11d024a44c896d1894b261a83,63
order_status,115037,7,delivered,112654
order_purchase_timestamp,115037,95515,2017-08-08 20:26:31,63
order_approved_at,115023,87916,2017-08-08 20:43:31,63
order_delivered_carrier_date,113852,78867,2017-08-10 11:58:14,63
order_delivered_customer_date,112653,93239,2017-08-14 12:46:18,63
order_estimated_delivery_date,115037,449,2017-12-20 00:00:00,644
customer_unique_id,115037,92939,9a736b248f67d166d2fbb006bcb877c3,75
customer_city,115037,4044,sao paulo,18224


## Data Cleaning

### Missing Values

In [27]:
sum_missing = data_raw.isna().sum()
percent_missing = round(data_raw.isnull().sum() *100 / len(data_raw),2)
missing_value_df = pd.DataFrame({'feature_name': data_raw.columns,
                                 'sum_missing': sum_missing,
                                 'percent_missing': percent_missing}).reset_index(drop=True).sort_values(by='sum_missing', ascending=False)
missing_value_df.head(9)

,feature_name,sum_missing,percent_missing
39,review_comment_title,101302,88.06
40,review_comment_message,66357,57.68
6,order_delivered_customer_date,2384,2.07
5,order_delivered_carrier_date,1185,1.03
4,order_approved_at,14,0.01
24,product_length_cm,1,0.00
25,product_height_cm,1,0.00
26,product_width_cm,1,0.00
23,product_weight_g,1,0.00


In [28]:
data_raw[(data_raw['order_approved_at'].isna())]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_geolocation_lat,customer_geolocation_lng,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,seller_zip_code_prefix,seller_city,seller_state,seller_geolocation_lat,seller_geolocation_lng,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
6098,e04abd8149ef81b95221e88f6ed9ab6a,2127dc6603ac33544953ef05ec155771,delivered,2017-02-18 14:40:00,NaN,2017-02-23 12:04:47,2017-03-01 13:25:33,2017-03-17 00:00:00,8a9a08c7ca8900a200d83cf838a07e0b,6708,cotia,SP,-23.585162,-46.841815,1,0e20a07ca1714df21f9b07ca3bf7c682,4e7c18b98d84e05cbae3ff0ff03846c2,2017-02-22 13:40:00,309.90,39.11,41.0,675.0,2.0,20800.0,75.0,40.0,40.0,small_appliances,14882,jaboticabal,SP,-21.250593,-48.317620,1,boleto,1,349.01,5d0fd5deb7e6a4f952cf63cc52cc18e3,4,NaN,"Muita demora, mas, tudo ok.\r\nTerezinha",2017-03-21 00:00:00,2017-03-21 17:35:02
19130,8a9adc69528e1001fc68dd0aaebbb54a,4c1ccc74e00993733742a3c786dc3c1f,delivered,2017-02-18 12:45:31,NaN,2017-02-23 09:01:52,2017-03-02 10:05:06,2017-03-21 00:00:00,91efb7fcabc17925099dced52435837f,93548,novo hamburgo,RS,-29.690980,-51.116358,1,2c2b6a28924791234bd386bddb17512e,a4b6b9b992b46e9ef863637af96e04bc,2017-02-22 11:45:31,379.00,17.86,56.0,498.0,1.0,1008.0,33.0,14.0,26.0,construction_tools_safety,88090,florianopolis,SC,-27.589021,-48.598946,1,boleto,1,396.86,a2f476daadcb4b1f28542a6eabf99612,5,NaN,NaN,2017-03-03 00:00:00,2017-03-04 23:25:49
26215,5cf925b116421afa85ee25e99b4c34fb,29c35fc91fc13fb5073c8f30505d860d,delivered,2017-02-18 16:48:35,NaN,2017-02-22 11:23:10,2017-03-09 07:28:47,2017-03-31 00:00:00,7e1a5ca61b572d76b64b6688b9f96473,62700,caninde,CE,-4.357711,-39.314913,1,c6dd917a0be2a704582055949915ab32,7a67c85e85bb2ce8582c35f2203ad736,2017-02-22 15:48:35,79.99,26.82,54.0,1012.0,1.0,1200.0,42.0,25.0,15.0,cool_stuff,3426,sao paulo,SP,-23.554079,-46.533242,1,boleto,1,106.81,4930703ae4fccd4d2d3b44cf63a9e632,5,NaN,NaN,2017-03-10 00:00:00,2017-03-13 00:18:46
26786,12a95a3c06dbaec84bcfb0e2da5d228a,1e101e0daffaddce8159d25a8e53f2b2,delivered,2017-02-17 13:05:55,NaN,2017-02-22 11:23:11,2017-03-02 11:09:19,2017-03-20 00:00:00,c8822fce1d0bfa7ddf0da24fff947172,27945,macae,RJ,-22.370400,-41.795154,1,c6dd917a0be2a704582055949915ab32,7a67c85e85bb2ce8582c35f2203ad736,2017-02-21 12:05:55,79.99,15.77,54.0,1012.0,1.0,1200.0,42.0,25.0,15.0,cool_stuff,3426,sao paulo,SP,-23.554079,-46.533242,1,boleto,1,95.76,4e44dd381f3c4f7afddacf8b9aa0c47a,5,NaN,NaN,2017-03-03 00:00:00,2017-03-04 00:06:38
31061,c1d4211b3dae76144deccd6c74144a88,684cb238dc5b5d6366244e0e0776b450,delivered,2017-01-19 12:48:08,NaN,2017-01-25 14:56:50,2017-01-30 18:16:01,2017-03-01 00:00:00,6ff8b0d7b35d5c945633b8d60165691b,11030,santos,SP,-23.981964,-46.310040,1,5ab02ca028398131a5ae91401eb49788,80e6699fe29150b372a0c8a1ebf7dcc8,2017-01-23 12:48:08,39.99,14.52,33.0,1322.0,2.0,700.0,26.0,16.0,21.0,sports_leisure,83323,pinhais,PR,-25.440714,-49.188444,1,boleto,1,54.51,a8353da830468c234d99152e0b714fed,4,NaN,Bom custo x benefício pra proteger o crânio da...,2017-02-01 00:00:00,2017-02-03 11:18:15
44373,d69e5d356402adc8cf17e08b5033acfb,68d081753ad4fe22fc4d410a9eb1ca01,delivered,2017-02-19 01:28:47,NaN,2017-02-23 03:11:48,2017-03-02 03:41:58,2017-03-27 00:00:00,2e0a2166aa23da2472c6a60c4af6f7a6,3573,sao paulo,SP,-23.561681,-46.506271,1,cae2e38942c8489d9d7a87a3f525c06b,cca3071e3e9bb7d12640c9fbe2301306,2017-02-26 01:28:47,149.80,13.63,54.0,586.0,1.0,1700.0,33.0,8.0,23.0,furniture_decor,14940,ibitinga,SP,-21.766477,-48.831547,1,boleto,1,163.43,6

This dataset contains 9 columns with missing values. The treatment is:
1. `review_comment_title, review_comment_message` contains too many missing values and a free text values, so it will be drop completely
2. `order_delivered_customer_date, order_delivered_carrier_date` contains date for each action, so it's reasonable if i contais empty rows, for analysis purpose we'll fill with '-'
3. For `order_approved_at` all rows with missing values turn out is an invalid values because each rows `order_status` are 'delivered' so we'll drop the missing value in this colums
4. For colums that contains `product descriptions` turn out to be on the same rows so we'll drop the rows

In [29]:
data_pre = data_raw.copy()

In [30]:
miss_drop = ['review_comment_title','review_comment_message']
data_pre = data_pre.drop(miss_drop, axis=1)

In [31]:
data_pre['order_delivered_customer_date'].fillna('-', inplace=True)
data_pre['order_delivered_carrier_date'].fillna('-', inplace=True)

In [32]:
data_raw[(data_raw['product_length_cm'].isnull())|(data_raw['product_height_cm'].isnull())|
         (data_raw['product_width_cm'].isnull())|(data_raw['product_weight_g'].isnull())]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_geolocation_lat,customer_geolocation_lng,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,seller_zip_code_prefix,seller_city,seller_state,seller_geolocation_lat,seller_geolocation_lng,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
44427,85f8ad45e067abd694b627859fa57453,1d088dea8732788ec35dd4ee6dd76112,delivered,2017-01-29 21:40:02,2017-01-30 22:32:20,2017-02-02 05:04:58,2017-02-07 10:46:00,2017-03-08 00:00:00,69b2b29382af03e03e486ad307a76000,90160,porto alegre,RS,-30.049304,-51.21316,1,09ff539a621711667c43eba6a3bd8466,8b8cfc8305aa441e4239358c9f6f2485,2017-02-03 21:40:02,1934.0,27.0,60.0,865.0,3.0,NaN,NaN,NaN,NaN,baby,86020,londrina,PR,-23.306635,-51.16455,1,credit_card,10,1961.0,125af5098b160c059cd60bafcc1f6ad1,5,NaN,NaN,2017-02-08 00:00:00,2017-02-09 10:09:24


In [33]:
data_pre.dropna(subset=['order_approved_at','product_length_cm'], how='all', inplace=True)

In [34]:
data_pre[(data_pre['order_status']== 'canceled')&(data_pre['order_delivered_carrier_date']!='-')]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_geolocation_lat,customer_geolocation_lng,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,seller_zip_code_prefix,seller_city,seller_state,seller_geolocation_lat,seller_geolocation_lng,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_creation_date,review_answer_timestamp
718,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,-,2018-02-22 00:00:00,c4ebedb09beb89cc0314c5c0c33f8053,9961,diadema,SP,-23.716539,-46.601640,1,a0b7d5a992ccda646f2d34e418fff5a0,95f83f51203c626648c875dd41874c7f,2018-02-01 21:58:39,69.90,26.11,45.0,67.0,2.0,12300.0,40.0,40.0,40.0,furniture_decor,37564,borda da mata,MG,-22.264094,-46.158564,1,credit_card,4,96.01,f242ce44c6572a94907a446b8fda32c8,1,2018-03-02 00:00:00,2018-03-05 01:28:24
719,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,-,2018-02-22 00:00:00,c4ebedb09beb89cc0314c5c0c33f8053,9961,diadema,SP,-23.716539,-46.601640,1,a0b7d5a992ccda646f2d34e418fff5a0,95f83f51203c626648c875dd41874c7f,2018-02-01 21:58:39,69.90,26.11,45.0,67.0,2.0,12300.0,40.0,40.0,40.0,furniture_decor,37564,borda da mata,MG,-22.264094,-46.158564,1,credit_card,4,96.01,105facb42b5b9147996f0b1b6fbf7b80,1,2018-02-24 00:00:00,2018-02-27 00:13:54
1239,3a129877493c8189c59c60eb71d97c29,0913cdce793684e52bbfac69d87e91fd,canceled,2018-01-25 13:34:24,2018-01-25 13:50:20,2018-01-26 21:42:18,-,2018-02-23 00:00:00,11c3d6c93dea6ce86769a4835c171bd9,87013,maringa,PR,-23.419864,-51.937051,1,7e857e35018e4a05b7d124d1628dc1f0,3fd1e727ba94cfe122d165e176ce7967,2018-01-31 13:50:20,35.90,15.10,14.0,1346.0,1.0,329.0,20.0,16.0,18.0,sports_leisure,14802,araraquara,SP,-21.760806,-48.172285,1,credit_card,1,14.12,15c8f3b3ff412d0f8d6b72053335fd0b,1,2018-02-25 00:00:00,2018-02-25 11:02:38
1240,3a129877493c8189c59c60eb71d97c29,0913cdce793684e52bbfac69d87e91fd,canceled,2018-01-25 13:34:24,2018-01-25 13:50:20,2018-01-26 21:42:18,-,2018-02-23 00:00:00,11c3d6c93dea6ce86769a4835c171bd9,87013,maringa,PR,-23.419864,-51.937051,1,7e857e35018e4a05b7d124d1628dc1f0,3fd1e727ba94cfe122d165e176ce7967,2018-01-31 13:50:20,35.90,15.10,14.0,1346.0,1.0,329.0,20.0,16.0,18.0,sports_leisure,14802,araraquara,SP,-21.760806,-48.172285,2,voucher,1,36.88,15c8f3b3ff412d0f8d6b72053335fd0b,1,2018-02-25 00:00:00,2018-02-25 11:02:38
3368,1950d777989f6a877539f53795b4c3c3,1bccb206de9f0f25adc6871a1bcf77b2,canceled,2018-02-19 19:48:52,2018-02-19 20:56:05,2018-02-20 19:57:13,2018-03-21 22:03:51,2018-03-09 00:00:00,6a04c8339c44b23e189446216a494569,32071,contagem,MG,-19.918634,-44.126762,1,e435ceb7ced9b8446defd858630454ed,2a61c271b1ac763450d846849783e922,2018-02-23 20:56:05,29.99,14.10,28.0,792.0,1.0,440.0,20.0,11.0,17.0,health_beauty,9175,santo andre,SP,-23.686456,-46.520612,1,credit_card,2,44.09,2c6741d2b72eb45a9ef713560f5eccf7,3,2018-03-11 00:00:00,2018-03-11 04:00:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108961,c9f06c1229ec7cd4ec9aed55d434b499,fa89053140d09081b3a3a4608a74c083,canceled,2018-02-05 20:26:29,2018-02-05 20:49:48,2018-02-06 19:49:32,-,2018-03-01 00:00:00,c7487ffbfa6bcd0d672d080467914d6a,14870,jaboticabal,SP,-21.253340,-48.328414,1,8c292ca193d326152e335d77176746f0,7e1fb0a3ebfb01ffb3a7dae98bf3238d,2018-02-09 20:49:48,150.00,12.55,60.0,2894.0,1.0,200.0,17.0,11.0,15.0,health_beauty,14403,franca,SP,-20.548228,-47.395897,1,credit

As we can see, there's 68 rows where `order_status` canceled but there's date for delivery, which is not possible because if the order canceled then it will be not shipped. So we'll drop rows with that particular conditions

In [35]:
condition = (data_pre['order_status']=='canceled') & (data_pre['order_delivered_carrier_date']!='-')
data_pre.drop(index=data_pre[condition].index, inplace=True)

## Customers Growth Analysis
Quarter-on-Quarter Customers Growth Analysis and Forecast

In [36]:
data_an1 = data_pre.copy()

### Extract Year and Quarter from order_purchase_timestamp

In [37]:
data_an1['order_purchase_timestamp'] = pd.to_datetime(data_pre['order_purchase_timestamp'])

In [38]:
data_an1['year_purchase'] = data_an1['order_purchase_timestamp'].dt.year
data_an1['quarter_purchase'] = data_an1['order_purchase_timestamp'].dt.quarter

In [39]:
user_monthly = data_an1.groupby(['year_purchase','quarter_purchase']).agg({'customer_unique_id':'count'}).reset_index()
user_monthly['average_monthly_user'] = np.where((user_monthly['year_purchase'] == 2016) & (user_monthly['quarter_purchase'] == 3), 
                                                 user_monthly['customer_unique_id'] / 1, user_monthly['customer_unique_id'] / 3).round(1)
user_monthly = user_monthly.drop('customer_unique_id',axis=1)
user_monthly = user_monthly.rename(columns={'year_purchase':'year','quarter_purchase':'quarter'})
user_monthly

,year,quarter,average_monthly_user
0,2016,3,3.0
1,2016,4,124.0
2,2017,1,2030.7
3,2017,2,3574.3
4,2017,3,4928.3
5,2017,4,6872.7
6,2018,1,8108.0
7,2018,2,7822.3
8,2018,3,4861.7


In [40]:
new_customer = data_an1.groupby(['year_purchase','quarter_purchase']).customer_unique_id.nunique().reset_index(name='new_customers')
new_customer = new_customer.rename(columns={'year_purchase':'year','quarter_purchase':'quarter'})
new_customer

,year,quarter,new_customers
0,2016,3,2
1,2016,4,294
2,2017,1,4895
3,2017,2,8739
4,2017,3,11878
5,2017,4,16787
6,2018,1,19942
7,2018,2,19351
8,2018,3,12390


In [41]:
repeat_customer = data_an1.groupby(['year_purchase','quarter_purchase','customer_unique_id']).size().reset_index(name='buy_freq')
repeat_customer = repeat_customer[repeat_customer['buy_freq'] > 1].groupby(['year_purchase','quarter_purchase']).agg({'customer_unique_id':'nunique'}).reset_index()
repeat_customer = repeat_customer.rename(columns={'year_purchase':'year','quarter_purchase':'quarter','customer_unique_id':'repeating_cust'})
repeat_customer

,year,quarter,repeating_cust
0,2016,3,1
1,2016,4,43
2,2017,1,711
3,2017,2,1265
4,2017,3,1767
5,2017,4,2415
6,2018,1,2837
7,2018,2,2679
8,2018,3,1480


In [42]:
# Merge all tables
cust_growth = user_monthly.merge(new_customer, on=['year','quarter'])
cust_growth = cust_growth.merge(repeat_customer, on=['year','quarter'])
cust_growth['time'] = cust_growth.apply(lambda x: '{} - Q{}'.format(int(x['year']), int(x['quarter'])), axis=1)
cust_growth

,year,quarter,average_monthly_user,new_customers,repeating_cust,time
0,2016,3,3.0,2,1,2016 - Q3
1,2016,4,124.0,294,43,2016 - Q4
2,2017,1,2030.7,4895,711,2017 - Q1
3,2017,2,3574.3,8739,1265,2017 - Q2
4,2017,3,4928.3,11878,1767,2017 - Q3
5,2017,4,6872.7,16787,2415,2017 - Q4
6,2018,1,8108.0,19942,2837,2018 - Q1
7,2018,2,7822.3,19351,2679,2018 - Q2
8,2018,3,4861.7,12390,1480,2018 - Q3


### Forecasting Customer Growth using Exponetial Smoothing

In [43]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [44]:
# menentukan jumlah periode yang akan di-forecast
forecast_periods = 5
# menyimpan data dari kolom yang akan di-forecast sebagai array
average_monthly_user_data = cust_growth['average_monthly_user'].values
new_customers_data = cust_growth['new_customers'].values
repeating_cust_data = cust_growth['repeating_cust'].values
# membuat model Exponential Smoothing
model_monthly = ExponentialSmoothing(average_monthly_user_data, trend='add', seasonal='add', seasonal_periods=3)
model_new = ExponentialSmoothing(new_customers_data, trend='add', seasonal='add', seasonal_periods=3)
model_rep = ExponentialSmoothing(repeating_cust_data, trend='add', seasonal='add', seasonal_periods=3)
# melakukan fitting model
fit_model1 = model_monthly.fit()
fit_model2 = model_new.fit()
fit_model3 = model_rep.fit()
# melakukan forecast
forecast_average_monthly_user = fit_model1.forecast(forecast_periods)
forecast_new_customers = fit_model2.forecast(forecast_periods)
forecast_repeating_cust = fit_model3.forecast(forecast_periods)

C:\Users\Adriana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [45]:
forecast_data = {'year':[2018,2019,2019,2019,2019],
                 'quarter':[4,1,2,3,4],
                 'average_monthly_user':forecast_average_monthly_user,
                 'new_customers':forecast_new_customers,
                 'repeating_cust':forecast_repeating_cust}
forecast_data = pd.DataFrame(forecast_data)
forecast_data['time'] = forecast_data.apply(lambda x: '{} - Q{}'.format(int(x['year']), int(x['quarter'])), axis=1)
forecast_data[['average_monthly_user', 'new_customers', 'repeating_cust']] = forecast_data[['average_monthly_user', 'new_customers', 'repeating_cust']].round(1)
forecast_data

,year,quarter,average_monthly_user,new_customers,repeating_cust,time
0,2018,4,7230.3,19069.9,2610.8,2018 - Q4
1,2019,1,6910.7,17705.3,2495.0,2019 - Q1
2,2019,2,7974.8,20632.7,2857.1,2019 - Q2
3,2019,3,8629.1,21725.4,3124.5,2019 - Q3
4,2019,4,8309.4,20360.8,3008.7,2019 - Q4


In [46]:
forecast_growth = pd.concat([cust_growth, forecast_data], ignore_index=True)
forecast_growth

,year,quarter,average_monthly_user,new_customers,repeating_cust,time
0,2016,3,3.0,2.0,1.0,2016 - Q3
1,2016,4,124.0,294.0,43.0,2016 - Q4
2,2017,1,2030.7,4895.0,711.0,2017 - Q1
3,2017,2,3574.3,8739.0,1265.0,2017 - Q2
4,2017,3,4928.3,11878.0,1767.0,2017 - Q3
5,2017,4,6872.7,16787.0,2415.0,2017 - Q4
6,2018,1,8108.0,19942.0,2837.0,2018 - Q1
7,2018,2,7822.3,19351.0,2679.0,2018 - Q2
8,2018,3,4861.7,12390.0,1480.0,2018 - Q3
9,2018,4,7230.3,19069.9,2610.8,2018 - Q4


In [64]:
fig = px.line(forecast_growth, x='time', y=['new_customers','average_monthly_user','repeating_cust'], title='Customer Growth Forecasting',
              template='plotly_dark',width=1400,height=800)
fig.data[0].line.smoothing = True
fig.data[0].line.shape = 'spline'
fig.data[0].line.color = 'teal'
fig.data[0].line.width = 4
fig.data[0].name = 'New Customers'
fig.data[1].line.smoothing = True
fig.data[1].line.shape = 'spline'
fig.data[1].line.color = 'orange'
fig.data[1].name = 'Average Monthly Users'
fig.data[1].line.width = 4
fig.data[2].line.smoothing = True
fig.data[2].line.shape = 'spline'
fig.data[2].line.color = 'red'
fig.data[2].name = 'Repeated Customers'
fig.data[2].line.width = 4
fig.add_vrect(x0=8.3, x1=13, annotation_text="Forecast", annotation_position="top left", annotation=dict(font_size=20),
              fillcolor="violet", opacity=0.25, line_width=0)
annotations = []
for i in range(1, len(forecast_growth)):
    total_new = forecast_growth.iloc[i]['new_customers']
    total_amu = forecast_growth.iloc[i]['average_monthly_user']
    total_rep = forecast_growth.iloc[i]['repeating_cust']
    quarters = forecast_growth.iloc[i]['time']
    total_new_before = forecast_growth.iloc[i-1]['new_customers']
    total_amu_before = forecast_growth.iloc[i-1]['average_monthly_user']
    total_rep_before = forecast_growth.iloc[i-1]['repeating_cust']
    if total_new != 0 and total_new_before!= 0:
        perc_new = (total_new - total_new_before) / total_new_before * 100
        if perc_new != 0:
            if perc_new >= 0:
                annotation_new = f'+{perc_new:.1f}%'
            else:
                annotation_new = f'{perc_new:.1f}%'
            annotations.append(dict(x=quarters, y=total_new, text=annotation_new, showarrow=True, arrowhead=2, ax=0, ay=-40))
    if total_amu != 0 and total_amu_before!= 0:
        perc_amu = (total_amu - total_amu_before) / total_amu_before * 100
        if perc_amu != 0:
            if perc_amu >= 0:
                annotation_amu = f'+{perc_amu:.1f}%'
            else:
                annotation_amu = f'{perc_amu:.1f}%'
            annotations.append(dict(x=quarters, y=total_amu, text=annotation_amu, showarrow=True, arrowhead=2, ax=0, ay=-40))
    if total_rep != 0 and total_rep_before!= 0:
        perc_rep = (total_rep - total_rep_before) / total_rep_before * 100
        if perc_rep != 0:
            if perc_rep >= 0:
                annotation_rep = f'+{perc_rep:.1f}%'
            else:
                annotation_rep = f'{perc_rep:.1f}%'
            annotations.append(dict(x=quarters, y=total_rep, text=annotation_rep, showarrow=True, arrowhead=2, ax=0, ay=40))
for annotation in annotations:
    fig.add_annotation(annotation)
fig.show()

### Customers Clustering